## GPS Spoofing Detection

### 1. load data and preprocess

In [ ]:
# Load Data
import utils
import os
import numpy as np
import config

A, B = utils.load_image_pairs(path=config.SWISS_1280x720)
assert A.shape[0]==B.shape[0]
n = A.shape[0]
print(A.shape, B.shape)

# Some configuration
#feature_map_file_name = './mid_product/features_suzhou_res34_eval.h5'#'features_suzhou_res50.h5'
feature_map_file_name = config.FULL_RESIZED_FEATURE
#dst_file_name = './mid_product/dst_suzhou_res34_eval.npy'# 'dst_suzhou_res50.npy' 
# feature_shape = (512, 18, 26) # SWISS, resnet-18/34
feature_shape = (512, 17, 34) # SUZHOU, 1280x720, resnet-18/34
# feature_shape = (2048, 17, 34) # SUZHOU, resnet-50

In [ ]:
# Preprocess data by doing transformation
import torch

#A = A.astype(np.float)/255.0
#B = B.astype(np.float)/255.0 #.transpose(0,3,1,2)
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
x_a = torch.from_numpy((A-mean)/std).permute(0,3,1,2).float()
x_b = torch.from_numpy((B-mean)/std).permute(0,3,1,2).float()
print(x_a.size(), x_b.size())

### 2. get feature maps

In [ ]:
import torchvision.models as models
from torch import nn

pretrained_model = models.resnet34(pretrained=True)
feature_extractor = nn.Sequential(*list(pretrained_model.children())[:-1])
feature_extractor.eval()
for param in feature_extractor.parameters():
    param.requires_grad = False

In [ ]:
# Generate feature map and save
import h5py

def h5_save(fname, f_a, f_b):
    '''save f_a and f_b as fname'''
    with h5py.File(fname,'w') as f:
        f.create_dataset('f_a', data=f_a)
        f.create_dataset('f_b', data=f_b)
    
def h5_read(fname):
    '''read fname and return f_a and f_b'''
    with h5py.File(fname,'r') as f:
        return f['f_a'][:], f['f_b'][:]

if not os.path.exists(feature_map_file_name):
    f_a = np.zeros((n,)+feature_shape)
    f_b = np.zeros((n,)+feature_shape)
    for i in range(n):
        print( "Generating feature maps of %d th pair."%(i) )
        a = feature_extractor(x_a[i:i+1,:,:,:])
        b = feature_extractor(x_b[i:i+1,:,:,:])
        f_a[i] = a.detach().numpy()
        f_b[i] = b.detach().numpy()
    h5_save(feature_map_file_name, f_a, f_b)
else:
    print("Feature maps file already exists, we just read it.")
    f_a, f_b = h5_read(feature_map_file_name)

In [ ]:
# compute distance between unpaird and paired images
print("A domain feature maps size:", f_a.shape)
dst=np.zeros((n,n))
for shift in range(n):
    for idx in range(n):
        a = f_a[idx:idx+1]
        b = f_b[(idx+shift)%n:(idx+shift)%n+1]
        dst[idx,shift] = np.linalg.norm(a - b)
        print('dst(idx,shift)(%d,%d)=%f' % (idx,shift,dst[idx,shift]))
np.save(dst_file_name, dst)

In [ ]:
# visualize
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
fig_size = [10,10]
plt.rcParams["figure.figsize"] = fig_size
plt.axis('equal')
ax = sns.heatmap(dst,
                 xticklabels=2,
                 yticklabels=2)
ax.set_xlabel('Shift')
ax.set_ylabel('Image Index')
ax.set_title('Distance Matrix')
plt.show()

### 3. analyse the feature maps

In [ ]:
print('Min, Max and Mean of Distances:')
print(np.min(dst), np.max(dst), np.average(dst))
lower, higher = np.min(dst), np.max(dst)

dst_t, dst_f = dst[:,0], dst[:,1:]
#print(dst_t.shape, dst_f.shape)
print('Min, Max and Mean of paired images:', np.min(dst_t), np.max(dst_t), np.mean(dst_t))
print('Min, Max and Mean of unpaired images:', np.min(dst_f), np.max(dst_f), np.mean(dst_f))

print('Sorted distance of paired images:')
print(np.sort(dst_t))

print('First n sorted distance of unpaired images:')
print(np.sort(dst_f.flatten())[:n])

In [ ]:
def predict(dst, threshold):
    return (dst <= threshold).astype(np.int)
def ground_truth(dst):
    n = dst.shape[0]
    gt = np.zeros((n,n)).astype(np.int)
    gt[:,0] = 1
    return gt
def confusion_matrix(pred, gt):
    n = gt.shape[0]
    TP = np.sum(gt[:,0] == pred[:,0])
    FN = np.sum(gt[:,0] != pred[:,0])
    TN = np.sum(gt[:,1:] == pred[:,1:])
    FP = np.sum(gt[:,1:] != pred[:,1:])
    TPR=TP/(TP+FN)
    FPR=FP/(FP+TN)
    ACC = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F1 = 2*precision*recall/(precision+recall)
    return TP,FP,TN,FN, TPR,FPR, ACC,precision,recall,F1

threshold = 465
pred = predict(dst, threshold)
gt = ground_truth(dst)

print("TP, FP, TN, FN, TPR, FPR, Accuracy, Precision, Recall, F1:")
print(confusion_matrix(pred, gt))
#%matplotlib inline
plt.axis('equal')
ax = sns.heatmap(pred,
                 xticklabels=2,
                 yticklabels=2)
ax.set_xlabel('Shift')
ax.set_ylabel('Image Index')
ax.set_title('Prediction Matrix')
plt.show()

In [ ]:
# Draw ROC curve
np.seterr(divide='ignore',invalid='ignore')
ROC_x, ROC_y = [0], [0]
for threshold in range(int(lower),int(higher)+1):
    pred = predict(dst, threshold)
    gt = ground_truth(dst)
    conf_mat = confusion_matrix(pred, gt)
    x, y = conf_mat[5], conf_mat[4] # x: FPR, y: TPR
    ROC_x.append(x)
    ROC_y.append(y)
ROC_x.append(1)
ROC_y.append(1)
#%matplotlib inline
plt.plot(ROC_x, ROC_y)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.axis('equal')
plt.show()